In [1]:
import requests, re
from bs4 import BeautifulSoup
from IPython.display import HTML, display

import pandas as pd
 
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer
# goes to page 460

# first page of thread:
# store data in lists --> pandas dataframe

# THREAD NAME
# <h3 class="post-title"><a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941">I can&#039;t live without my dad - anyone feel the same?</a></h3>

# THREAD ADDRESS
# page_link

# USERNAME
# <div class="field field-name-ds-user-picture field-type-ds field-label-hidden">
# <div class="field-items"><div class="field-item even">
# <a href="/about-cancer/cancer-chat/users/paulus">

# POST DATE
# <span class="post-created hidden-xs">22 Jun 2020 16:35</span>

# POST TEXT
# <div class="field field-name-field-harmony-text field-type-text-long field-label-hidden">

# POST IS A REPLY TO POST_ID
# <span class="post-is-reply-to">22 Jun 2020 21:04 in response to <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941" class="active">bras1548</a></span>

# POST ID
# <a name="post-1305941"></a>


#### FUNCTION: SCRAPE A THREAD PAGE

In [2]:
# FUNCTION: SCRAPE A THREAD PAGE
def page_scraper_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')
    
    # THREAD ADDRESS 
    thread_address_list = []
    thread_address = soup.find("link",{"rel":"canonical"})['href']
    thread_address = page_link
    #print(thread_address)

    # THREAD NAME
    thread_name_list = []
    thread_group = soup.find("h3", {"class": "post-title"})
    thread_name = thread_group.find("a").text
    #print(thread_name)

    # POST ID
    post_id_list = []
    post_id_group = soup.findAll("a", {"name": re.compile(r".*")})

    for post_id_g in post_id_group:
        if "post-" in post_id_g["name"]:
            post_id_list.append(post_id_g["name"])
            #print(post_id_g["name"])

    # POST USERNAME
    username_list = []
    user_group = soup.findAll("div", {"class": "field field-name-ds-user-picture field-type-ds field-label-hidden"})
    
    for user_g in user_group:    
        username = user_g.find("a")["href"]
        username = username.replace("/about-cancer/cancer-chat/users/", "")
        username_list.append(username)

        # populate THREAD ADDRESS, THREAD NAME lists
        thread_address_list.append(thread_address)
        thread_name_list.append(thread_name)
        #print(username)

    # POST DATE
    date_list = []
    date_group = soup.findAll("span", {"class": "post-created hidden-xs"})

    for date_g in date_group:
        date = date_g.text
        date_list.append(date)
        #print(date)

    # POST TEXT
    text_list = []
    text_group = soup.findAll("div", {"class": "field field-name-field-harmony-text field-type-text-long field-label-hidden"})

    for text_g in text_group:
        text_tag = text_g.find("div").find("div")
        text = text_tag.text
        text_list.append(text)
        #print(text_tag.text, "\n")

    # POST IS A REPLY TO
    reply_to_list = []
    reply_group = soup.findAll("span", {"class": "post-is-reply-to"})

    # add "N/A only to first posts in thread
    
    reply_group_len = len(reply_group)
    text_group_len = len(text_group)
    
    while reply_group_len < text_group_len:  # if CHANGED TO while
        reply_to_list.append("N/A")
        reply_group_len += 1
        
    for reply_g in reply_group:
        reply_to_id = reply_g.find("a")['href']
        reply_to_id = "https://www.cancerresearchuk.org" + reply_to_id
        reply_to_id = reply_to_id.replace(thread_address + "#", "")
        reply_to_list.append(reply_to_id)
        #print(reply_to_id)

    # COLUMN LISTS INTO A LIST OF ROWS
    new_rows = []

    for j in range(len(thread_name_list)):
        row_list = [thread_name_list[j], thread_address_list[j], username_list[j],
                    post_id_list[j], date_list[j], text_list[j], reply_to_list[j]]

        new_rows.append(row_list)
    
    return new_rows



#### FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

In [3]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

# <li class="cr-pagination__next">
# <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=1">Next</a></li>

def next_page_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')

    next_link = soup.find("li", {"class":"cr-pagination__next"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org/" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road?page=1"))
print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"))



None
https://www.cancerresearchuk.org//about-cancer/cancer-chat/thread/the-end-of-the-road?page=1


In [4]:
# TEST PAGE SCRAPER FUNCTION

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"
new_rows = page_scraper_cancer_research(page_link)


In [5]:
for row in new_rows:
    print(row[0])
    print(row[1])
    print(row[2])
    print(row[3])
    print(row[4])
    print(row[5])
    print(row[6], "\n")

The end of the road
https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road
goodhubby
post-1458971
19 Dec 2020 10:49
I always hoped I would never be writing this but my wife was given the news last week that her metastatic breast cancer was no longer treatable. Seven years ago she was diagnosed with stage 1b HER + tumour in her right breast which was surgically removed followed by radiotherapy and chemo, immunotherapy etc then at 4 years 10 months she was diagnosed with brain mets which were removed followed by radio therapy an chemo again, this year in April the scans showed liver mets and a small one in her lung. The hard part for me was to watch her look so beautiful again and then have to go through the same over again. The liver mets would not respond to any chemo of which 3 were tried and it was only when my wife begged the consultant to try another that she was offered it but we were told that it won't work due to the last few not working.  Her liver

#### FUNCTION: SCRAPE ONE THREAD

In [6]:
# FUNCTION: SCRAPE ONE THREAD

row_list = []
page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-long-does-he-have-left-colon-cancer"

def thread_scraper_cancer_research(page_link):

    new_rows = page_scraper_cancer_research(page_link)
    next_link = next_page_cancer_research(page_link)
    
    #for row in new_rows:  # REMOVE THIS LATER
    #    print(row[0])
    #    print(row[1])
    #    print(row[2])
    #    print(row[3])
    #    print(row[4])
    #    print(row[5])
    #    print(row[6], "\n")
        
    
    if next_link is not None:
        new_rows.extend(thread_scraper_cancer_research(next_link))

    return new_rows

In [8]:
# TEST: SCRAPE ONE THREAD

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-long-does-he-have-left-colon-cancer"

new_rows = thread_scraper_cancer_research(page_link)
print(len(new_rows))

1


#### FUNCTION: GET THREAD LINKS FROM PAGE

In [9]:
# FUNCTION: GET THREAD LINKS FROM PAGE

forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer"

def thread_link_scraper_cancer_research(forum_link):

    thread_links = []
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

    # <div class="tl-cell views-field views-field-nothing" >
    # <a href="/about-cancer/cancer-chat/thread/has-anyone-tried-bereavement-counselling">

    thread_link_group = soup.find_all("div", {"class":"tl-cell views-field views-field-nothing"})
    
    for thread_link_gr in thread_link_group:    
        new_thread_link = thread_link_gr.find('a')['href']
        thread_links.append("https://www.cancerresearchuk.org" + new_thread_link)
        #print("https://www.cancerresearchuk.org" + new_thread_link, "\n")
        
    return thread_links
    


#### FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

In [10]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

def next_forum_page_cancer_research(forum_link):
    
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

# <li class="next last"><a title="Go to next page" href="
    next_link = soup.find("li", {"class":"next last"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer"))
print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=460"))



https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=1
None


#### FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

In [11]:
# FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

def thread_links_cancer_research(forum_link):
    
    thread_links = []
    current_page = forum_link
    new_links = thread_link_scraper_cancer_research(current_page)
    thread_links.extend(new_links)
    
    next_link = next_forum_page_cancer_research(current_page)
    print(next_link)
    
    if next_link is not None:
        thread_links.extend(thread_links_cancer_research(next_link))
        
    return thread_links




In [12]:
# TEST FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer"
thread_links = thread_links_cancer_research(forum_link)


https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=1
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=2
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=3
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=4
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=5
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=6
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=7
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=8
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-s

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=73
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=74
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=75
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=76
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=77
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=78
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=79
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=80
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/cari

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=144
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=145
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=146
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=147
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=148
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=149
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=150
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=151
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-can

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=215
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=216
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=217
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=218
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=219
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=220
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=221
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=222
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-can

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=286
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=287
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=288
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=289
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=290
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=291
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=292
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=293
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-can

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=357
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=358
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=359
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=360
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=361
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=362
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=363
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=364
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-can

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=428
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=429
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=430
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=431
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=432
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=433
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=434
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/caring-for-someone-with-cancer?page=435
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-can

In [14]:
# OPEN FILE WITH SAVED 'LIVING WITH CANCER' LINKS

thread_links = pd.read_csv(r'/Users/Sofia/Desktop/cancer_research_caring_links.csv', delimiter="\t")
thread_links = thread_links["0"].values.tolist()
print(len(thread_links))


#test_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#for i in test_list[0:3]:
#    print(i)
    
#print("\n")

#for i in test_list[3:6]:
#    print(i)
    
#print("\n")

#for i in test_list[6:]:
#    print(i)

4610


In [108]:
# DEBUGGING PROBLEM PAGE

bug_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/confused-30"
#bug_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/seeing-signs-from-deceased-loved-ones?page=7"

bug_rows = thread_scraper_cancer_research(bug_link)
#bug_rows = page_scraper_cancer_research(bug_link)

In [110]:
#for line in bug_rows:
#    print(line[0])
#    print(line[1])
#    print(line[2])
#    print(line[3])
#    print(line[4])
#    print(line[5])
#    print(line[6], "\n")

In [39]:
# RUN 'SCRAPE THREAD' ON THREAD_LINKS

#caring_data = [] # DO NOT RESET !!

count = 4425

for thread in thread_links[4424:]:
    
    print(count, thread)

    new_rows = thread_scraper_cancer_research(thread)    
    caring_data.extend(new_rows)
    
    count += 1



4425 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/need-advise-please-x
4426 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mother-in-law-has-aml
4427 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mum-has-lost-interest
4428 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/cervical-cancer-20
4429 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/withdrawing
4430 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-to-support
4431 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dad-has-stage-4-cancer-in-the-lung
4432 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-dont-know-what-to-expect-with-my-nans-brain-cancer
4433 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-daughter-has-acute-lymphoblastc-leukaemia
4434 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lung-cancer-48
4435 https://www.cancerresearchuk.org/abo

4509 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/stepdad-has-been-diagnosed-with-stomach-cancer
4510 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mum-scared-so-am-i
4511 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/do-i-delay-my-studies
4512 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mums-cancer-talking-about-death
4513 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/new-to-grouplooking-for-others-with-epithelioid-sarcoma
4514 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/any-single-parents-to-a-child-with-cancer
4515 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mum-diagnosed-with-colon-cancer
4516 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-has-gone-really-down-hill-in-the-4-weeks-since-his-xray/preliminary-diagnosis-is-this
4517 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-to-support-mum-wi

4587 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-feel-so-weak-as-a-mother
4588 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-3-year-old-has-cancer-and-what-im-going-to-do-about-it
4589 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-to-cope-as-a-parent-when-your-teenager-has-just-been-diagnosed-with-cancer
4590 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/baby-first-chemo-session
4591 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/baby-daughter-has-neuroblastoma-in-stomach-would-like-to-hear-from-others-personal-stories
4592 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/baby-with-cancer-anyone-else
4593 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/cousin-recently-died-of-cancer-he-was-only-7
4594 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lets-beat-kids-cancer
4595 https://www.cancerresearchuk.org/about-cancer/cancer-chat/t

In [41]:
# SAVE PRELIMINARY DATA

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_caring_prelim = pd.DataFrame(caring_data, columns = column_names)  

cancer_research_caring_prelim.to_csv(r'/Users/Sofia/Desktop/cancer_research_caring.csv', sep ='\t', index = False, header=True)




In [40]:
print(len(caring_data))

# print(len(thread_links))
    
#for line in caring_data:
#    print(line[0])
#    print(line[1])
#    print(line[2])
#    print(line[3])
#    print(line[4])
#    print(line[5])
#    print(line[6], "\n")


21122


In [13]:
# SAVE 'CARING FOR SOMEONE WITH CANCER' FORUM LINKS TO FILE

#print(len(thread_links))

#for thread in thread_links:
#    print(thread)

cancer_research_caring_links = pd.DataFrame(thread_links)
cancer_research_caring_links.to_csv(r'/Users/Sofia/Desktop/cancer_research_caring_links.csv', sep ='\t', index = False, header=True)



In [137]:
# GLOBAL database for 
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_coping_w_loss = pd.DataFrame(new_rows, columns = column_names)  
print (cancer_research_coping_w_loss)

cancer_research_coping_w_loss.to_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss_test.csv', sep ='\t', index = False, header=True)


                                          Thread_name  \
0   I can't live without my dad - anyone feel the ...   
1   I can't live without my dad - anyone feel the ...   
2   I can't live without my dad - anyone feel the ...   
3   I can't live without my dad - anyone feel the ...   
4   I can't live without my dad - anyone feel the ...   
5   I can't live without my dad - anyone feel the ...   
6   I can't live without my dad - anyone feel the ...   
7   I can't live without my dad - anyone feel the ...   
8   I can't live without my dad - anyone feel the ...   
9   I can't live without my dad - anyone feel the ...   
10  I can't live without my dad - anyone feel the ...   
11  I can't live without my dad - anyone feel the ...   
12  I can't live without my dad - anyone feel the ...   
13  I can't live without my dad - anyone feel the ...   
14  I can't live without my dad - anyone feel the ...   
15  I can't live without my dad - anyone feel the ...   
16  I can't live without my dad